# **Import required files**

In [ ]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image

# **Load ResNet34 model**

In [ ]:
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# **Uploading files**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving imagenet_classes.txt to imagenet_classes.txt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dalgom.jpg to Dalgom.jpg
Saving patched_dalgom.jpg to patched_dalgom.jpg


# Image **preprocessing**

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# **Initialize adversarial patch**

In [ ]:
patch_size = (50, 50)
ad_patch = torch.rand(3, *patch_size, requires_grad=True)

# Load ImageNet classes
with open('imagenet_classes.txt') as f:
    class_labels = [line.strip() for line in f.readlines()]

# Set target class (Pomeranian, 260)
target_class_idx = 260

# **Load and preprocess image**

In [ ]:
image = preprocess(Image.open('Dalgom.jpg')).unsqueeze(0)

# **Optimizer**

In [ ]:
optimizer = torch.optim.Adam([ad_patch], lr=0.01)

# **Training loop**

In [ ]:
for _ in range(100):
    optimizer.zero_grad()
    patched_image = image.clone()
    patched_image[:, :, :patch_size[0], :patch_size[1]] = ad_patch
    output = model(patched_image)
    loss = -output[0, target_class_idx]
    loss.backward()
    optimizer.step()

# **Get the predicted class**

In [ ]:
_, predicted_class_idx = torch.max(output, 1)
predicted_class_idx = predicted_class_idx.item()

# **Print predicted class label**

In [ ]:
print(f'Predicted class: {class_labels[predicted_class_idx]}')

Predicted class: chow


# **Top 5 predictions with probabilities**

In [ ]:
softmax_probs = torch.nn.functional.softmax(output[0], dim=0)
top5_probs, top5_class_indices = torch.topk(softmax_probs, 5)

print("Top 5 predicted classes:")
for i in range(5):
    print(f"{class_labels[top5_class_indices[i]]}: {top5_probs[i].item() * 100:.2f}%")


Top 5 predicted classes:
chow: 87.67%
teddy: 4.33%
standard poodle: 3.22%
toy poodle: 1.68%
Pomeranian: 0.73%
